# Params for Simple_RNN

$params = cell\times input\_size + cell^2$

$cell^2$ means cells are fullly connected to themselves   meaning?


In [0]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow import keras
import numpy as np

In [356]:
cell = 1
input_size = 2
time_steps = 1    # doesn't change the number of params

model = keras.Sequential([
        SimpleRNN(cell, activation='relu', use_bias=False, input_shape=(time_steps, input_size)),
        #Dense(1, activation='relu', use_bias=False)    
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_61 (SimpleRNN)    (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [357]:
model.get_weights()

[array([[-0.21311605],
        [-1.3135166 ]], dtype=float32), array([[1.]], dtype=float32)]

In [0]:
model.set_weights([[[2],[3]],[[5]]])

In [359]:
x = np.array([1, 0])
x = x.reshape(1,1,2)
model.predict(x)

array([[2.]], dtype=float32)

# Add a time step

In [360]:
cell = 1
input_size = 2
time_steps = 2    # doesn't change the number of params

model = keras.Sequential([
        SimpleRNN(cell, activation='relu', use_bias=False, input_shape=(time_steps, input_size)),
        #Dense(1, activation='relu', use_bias=False)    
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_62 (SimpleRNN)    (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.set_weights([[[2],[3]],[[5]]])

In [362]:
x = np.array([1, 0, 0, 0])
x = x.reshape(1,2,2)
print(x)
model.predict(x)

[[[1 0]
  [0 0]]]


array([[10.]], dtype=float32)

In [363]:
x = np.array([1, 0, 1, 0])
x = x.reshape(1,2,2)
print(x)
model.predict(x)

[[[1 0]
  [1 0]]]


array([[12.]], dtype=float32)

In [364]:
x = np.array([1, 0, 0, 1])
x = x.reshape(1,2,2)
print(x)
model.predict(x)

[[[1 0]
  [0 1]]]


array([[13.]], dtype=float32)

# Add another timestep

In [365]:
cell = 1
input_size = 2
time_steps = 3    # doesn't change the number of params

model = keras.Sequential([
        SimpleRNN(cell, activation='relu', use_bias=False, input_shape=(time_steps, input_size)),
        #Dense(1, activation='relu', use_bias=False)    
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_63 (SimpleRNN)    (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.set_weights([[[2],[3]],[[5]]])

In [367]:
x = np.array([1, 0, 0, 1, 7, 11])
x = x.reshape(1,3,2)
print(x)
model.predict(x) == 2 * 7 + 3 * 11 + 5 * 13

[[[ 1  0]
  [ 0  1]
  [ 7 11]]]


array([[ True]])

# Add another input

In [368]:
x = np.array([1, 0, 0, 1, 7, 11, 1, 0, 0, 1, 7, 11])
x = x.reshape(2,3,2)
print(x)
model.predict(x) == 2 * 7 + 3 * 11 + 5 * 13

[[[ 1  0]
  [ 0  1]
  [ 7 11]]

 [[ 1  0]
  [ 0  1]
  [ 7 11]]]


array([[ True],
       [ True]])

# Prepare Training Data

In [369]:
def get_bit(n, bit):
    a = []
    for i in range(bit):
        a.append(n & 1)
        n >>= 1
    return a
  
for i in range(3):
    print(get_bit(i, 8))

[0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]


In [0]:
time_steps = 8

x = []
y = []
for i in range(2**time_steps):
    for j in range(2**time_steps):
        i_b = get_bit(i, time_steps)
        j_b = get_bit(j, time_steps)
        x.append([[i_b[i], j_b[i]] for i in range(time_steps)])
        y.append([[k] for k in get_bit(i+j, 8)])
    

In [371]:
i = 256*11 + 5 # 256*3 + 5
print(x[i])
print(y[i])

[[1, 1], [1, 0], [0, 1], [1, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0], [0], [0], [0], [1], [0], [0], [0]]


In [0]:
#for i in range(len(x)):
#  print(x[i], y[i])

In [373]:
import numpy as np

x = np.array(x)
x.shape

(65536, 8, 2)

In [374]:
y = np.array(y)
y.shape

(65536, 8, 1)

# Train

In [375]:
cell = 8
input_size = 2
time_steps = 8    # doesn't change the number of params

model = keras.Sequential([
        SimpleRNN(cell, activation='relu', return_sequences=True, use_bias=False, input_shape=(time_steps, input_size)),
        Dense(1, activation='relu', use_bias=False)    
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_64 (SimpleRNN)    (None, 8, 8)              80        
_________________________________________________________________
dense_41 (Dense)             (None, 8, 1)              8         
Total params: 88
Trainable params: 88
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['accuracy'])

In [377]:
model.fit(x, y, epochs=5, batch_size=16)  

Epoch 1/5
65536/65536 [==============================] - 11s 173us/step - loss: 0.0792 - acc: 0.9019
Epoch 2/5
65536/65536 [==============================] - 9s 130us/step - loss: 4.9192e-05 - acc: 1.0000
Epoch 3/5
65536/65536 [==============================] - 9s 132us/step - loss: 2.3435e-05 - acc: 1.0000
Epoch 4/5
65536/65536 [==============================] - 9s 139us/step - loss: 1.3780e-05 - acc: 1.0000
Epoch 5/5
65536/65536 [==============================] - 9s 136us/step - loss: 8.7074e-06 - acc: 1.0000


In [378]:

count = 0


for i in range(len(x)):
  xi = x[i].reshape(1, 8, 2)
  yi = model.predict(xi).reshape(8, 1)
  for j in range(len(yi)):
    if y[i][j][0] != (0 if yi[j][0] < 0.5 else 1):
      count += 1
      break
count




0